In [ ]:
import math
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import ray

import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import numpy as np
import time

from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid

import torch
import torch.nn.functional as F

from torch_geometric.nn import GCNConv

import matplotlib.pyplot as plt

import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing

from typing import Union, Tuple
from torch_geometric.typing import OptTensor, OptPairTensor, Adj, Size

from torch import Tensor
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.conv import MessagePassing

from ROOT import TLorentzVector

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render
from ax.utils.tutorials.cnn_utils import train, evaluate

In [ ]:
data = torch.from_numpy( np.load('/data/github/data/diHiggs_kin_train_data.npy').astype(np.float32))
data2 = torch.from_numpy( np.load('/data/github/data/diHiggs_kin_test_data.npy').astype(np.float32))

In [ ]:
e1_, e2_ = [], []
for i in range(6):
    for j in range(6):
        e1_.append(i)
        e2_.append(j)
        
def graph_trans(d, y=1, e1=e1_, e2=e2_):
    b1, b2, l1, l2, nu1, nu2 = TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector(), TLorentzVector()
    b1.SetPxPyPzE(d[17], d[18], d[19], d[20])
    b2.SetPxPyPzE(d[21], d[22], d[23], d[24])
    l1.SetPxPyPzE(d[25], d[26], d[27], d[28])
    l2.SetPxPyPzE(d[29], d[30], d[31], d[32])
    nu1.SetPxPyPzE(d[33], d[34], d[35], d[36])
    nu2.SetPxPyPzE(d[37], d[38], d[39], d[40])

    x = torch.tensor([[0, 1, 0, b1.Pt()/500., b1.E()/500., b1.M()],
                     [0, -1, 0, b2.Pt()/500., b2.E()/500., b2.M()],
                     [1, 0, 0, l1.Pt()/200., l1.E()/200., l1.M()],
                     [-1, 0, 0, l2.Pt()/200., l2.E()/200., l2.M()],
                     [0, 0, 1, nu1.Pt()/50., nu1.E()/50., nu1.M()],
                     [0, 0, -1, nu2.Pt()/50., nu2.E()/50., nu2.M()]], dtype=torch.float)

    edge_attr = []
    e1_, e2_ = [], []
    for i in range(6):
        edge_attr_i = []
        for j in range(6):
            e1_.append(i)
            e2_.append(j)
            edge_attr_i.append([b1, b2, l1, l2, nu1, nu2][i].DeltaR([b1, b2, l1, l2, nu1, nu2][j]))
        edge_attr.append(edge_attr_i)

    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    GNN_index = torch.tensor([e1, e2], dtype=torch.long)


    graph = Data(x=x, y=[y, y, y, y, y, y], GNN_index=GNN_index, edge_attr=edge_attr)
    return graph

In [ ]:
train_graph_list = []
for idx, d in enumerate(data):
    if(idx<=19000):
        y = 1
    else:
        y = 0
    train_graph_list.append(graph_trans(d, y))

test_graph_list = []
for idx, d in enumerate(data2):
    if(idx<=5000):
        y = 1
    else:
        y = 0
    test_graph_list.append(graph_trans(d, y))
    
    
train_loader = DataLoader(train_graph_list, 
                          batch_size=109,
                          shuffle=True)

test_loader = DataLoader(test_graph_list, batch_size=128, shuffle=False)

In [ ]:
class GNNConv0(MessagePassing):
    def __init__(self, in_channels, hidden1, hidden2, out_channels):
        super().__init__(aggr='add')
        self.mlp0 = Seq(Linear(in_channels, hidden1, bias=True),
                               ReLU())
        self.mlp1 = Seq(Linear(hidden1+1, hidden2, bias=True),
                       ReLU())
        self.mlp2 = Seq(Linear(hidden2+hidden1, out_channels, bias=True),
                       ReLU())


    def forward(self, x, GNN_index, edge_attr): 
        x = self.mlp0(x)
        out = self.propagate(GNN_index, x=x, edge_attr=edge_attr)
        out = torch.cat([x, out], dim=1)
        out = self.mlp2(out)
        return out

    def message(self, x_i, x_j, edge_attr_j):
        edge_info = edge_attr_j[:, i]
        edge_info = edge_info[:, None]
        x_j = torch.cat([x_j, edge_info], dim=1)
        x_j = self.mlp1(x_j)
        return x_j

In [ ]:
class GNNConv(MessagePassing):
    def __init__(self, in_channels, hidden, out_channels):
        super().__init__(aggr='add')
        self.mlp1 = Seq(Linear(in_channels+1, hidden, bias=True),
                       ReLU())
        self.mlp2 = Seq(Linear(hidden+in_channels, out_channels, bias=True),
                       ReLU())


    def forward(self, x, GNN_index, edge_attr):        
        out = self.propagate(GNN_index, x=x, edge_attr=edge_attr)
        out = torch.cat([x, out], dim=1)
        out = self.mlp2(out)
        return out

    def message(self, x_i, x_j, edge_attr_j):
        edge_info = edge_attr_j[:, i]
        edge_info = edge_info[:, None]
        x_j = torch.cat([x_j, edge_info], dim=1)
        x_j = self.mlp1(x_j)
        return x_j

In [ ]:
class MPNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GNNConv0(6, 58, 64, 116)
        self.conv2 = GNNConv(116, 4, 30)
        self.conv3 = GNNConv(30, 107, 2)

    def forward(self, data):
        x, GNN_index = data.x, data.GNN_index
        edge_attr = data.edge_attr

        x = self.conv1(x, GNN_index, edge_attr)
        x = self.conv2(x, GNN_index, edge_attr)
        x = self.conv3(x, GNN_index, edge_attr)

        return F.softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MPNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=3.588112521749169e-05,
                             betas=(0.9370461057934745, 0.9911704693310439),
                             weight_decay=0.010816528466771117,
                            )
scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma=0.9409084329632221
)

In [ ]:
def train(model, train_loader):
    model.train()
    loss_val = []

    for batch in train_loader:
        optimizer.zero_grad()
        
        label = torch.tensor(np.concatenate(batch.y)).to(device)
        out = model(batch.to(device))

        loss = criterion(out, label)
        
        loss.backward()
        optimizer.step()

        loss_val.append(loss.item())
        
    scheduler.step()

    loss = np.mean(loss_val)
    
    print(f'\n Train Epoch: {epoch} \tLoss: {loss:.6f}')
    
    return loss

In [ ]:
def test(model, test_loader):
    
    DNN_score1 = open('/data/github/result/MPNN_DNN_score1.TXT', 'a')    
    DNN_score2 = open('/data/github/result/MPNN_DNN_score2.TXT', 'a')

    model.eval()
    loss_val = []
    for idx, batch in enumerate(test_loader):
        out = model(batch.to(device)) 
        loss = criterion(out, torch.tensor(np.concatenate(batch.y)).to(device))
        
        loss_val.append(loss.item())

        out = (out.cpu().detach().numpy())[:]
        if(idx==0):
            test_out = out[:]
            test_y = np.array(batch.y)[:]
        else:
            test_out = np.concatenate((test_out[:], out[:]), axis=0)
            test_y = np.concatenate((test_y[:], np.array(batch.y)[:]), axis=0)


    for n in range(0,len(test_out)):
        DNN_score1.write(str((test_out[:,0:1][n].item())) + ' ' + str((test_out[:,1:2][n].item())))
        DNN_score1.write('\n')
        
    for n in range(0,len(test_y)):
        DNN_score2.write(str((test_y[:,0:1][n].item())) + ' ' + str((test_y[:,1:2][n].item())) + ' ' + str((test_y[:,2:3][n].item())) + ' ' + str((test_y[:,3:4][n].item())) + ' ' + str((test_y[:,4:5][n].item())) + ' ' + str((test_y[:,5:6][n].item())))
        DNN_score2.write('\n')
    
    loss = np.mean(loss_val)

    print(f'\nTest set: Loss: {loss:.6f}\n')
    print('----------------------------------------------------')

    return loss

In [ ]:
def Like(s, b, u, n):
    return math.e**((n*s+b)*math.log(u*s+b)-math.lgamma(n*s+b+1)-(u*s+b))

def IndiLikeRatioDis(NS1, NB1):
    return math.sqrt(-2*math.log((Like(NS1, NB1, 0.0, 1.0))/(Like(NS1, NB1, 1.0, 1.0))))

In [ ]:
DNN_score1 = open('/data/github/result/MPNN_DNN_score1.TXT', 'w')    
DNN_score2 = open('/data/github/result/MPNN_DNN_score2.TXT', 'w')

Train_Loss = []
Test_Loss = []

epochs = 1
best_sig = 0

for epoch in range(1, epochs + 1):
    start=time.time()
    print('########### Training epoch {} start ###########'.format(epoch))
    
    train_loss = train(model, train_loader)
    test_loss = test(model, test_loader)

    Train_Loss.append(train_loss)
    Test_Loss.append(test_loss)
    
    epoch_ = epoch

    Results1 = np.loadtxt('/data/github/result/MPNN_DNN_score1.TXT')
    Results2 = np.loadtxt('/data/github/result/MPNN_DNN_score2.TXT')

    test_out = Results1[151500*(epoch_-1):151500*(epoch_)]
    test_y = Results2[25250*(epoch_-1):25250*(epoch_)]
    
    Results_hh_1=test_out[:,1:2][:25250][:5000]
    Results_tt_1=test_out[:,1:2][:25250][5000:5000+19000]
    Results_tw_1=test_out[:,1:2][:25250][5000+19000:5000+19000+800]
    Results_tth_1=test_out[:,1:2][:25250][5000+19000+800:5000+19000+800+100]
    Results_ttv_1=test_out[:,1:2][:25250][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_1=test_out[:,1:2][:25250][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_1=test_out[:,1:2][:25250][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    Results_hh_2=test_out[:,1:2][25250:25250*2][:5000]
    Results_tt_2=test_out[:,1:2][25250:25250*2][5000:5000+19000]
    Results_tw_2=test_out[:,1:2][25250:25250*2][5000+19000:5000+19000+800]
    Results_tth_2=test_out[:,1:2][25250:25250*2][5000+19000+800:5000+19000+800+100]
    Results_ttv_2=test_out[:,1:2][25250:25250*2][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_2=test_out[:,1:2][25250:25250*2][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_2=test_out[:,1:2][25250:25250*2][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    Results_hh_3=test_out[:,1:2][25250*2:25250*3][:5000]
    Results_tt_3=test_out[:,1:2][25250*2:25250*3][5000:5000+19000]
    Results_tw_3=test_out[:,1:2][25250*2:25250*3][5000+19000:5000+19000+800]
    Results_tth_3=test_out[:,1:2][25250*2:25250*3][5000+19000+800:5000+19000+800+100]
    Results_ttv_3=test_out[:,1:2][25250*2:25250*3][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_3=test_out[:,1:2][25250*2:25250*3][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_3=test_out[:,1:2][25250*2:25250*3][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    Results_hh_4=test_out[:,1:2][25250*3:25250*4][:5000]
    Results_tt_4=test_out[:,1:2][25250*3:25250*4][5000:5000+19000]
    Results_tw_4=test_out[:,1:2][25250*3:25250*4][5000+19000:5000+19000+800]
    Results_tth_4=test_out[:,1:2][25250*3:25250*4][5000+19000+800:5000+19000+800+100]
    Results_ttv_4=test_out[:,1:2][25250*3:25250*4][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_4=test_out[:,1:2][25250*3:25250*4][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_4=test_out[:,1:2][25250*3:25250*4][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    Results_hh_5=test_out[:,1:2][25250*4:25250*5][:5000]
    Results_tt_5=test_out[:,1:2][25250*4:25250*5][5000:5000+19000]
    Results_tw_5=test_out[:,1:2][25250*4:25250*5][5000+19000:5000+19000+800]
    Results_tth_5=test_out[:,1:2][25250*4:25250*5][5000+19000+800:5000+19000+800+100]
    Results_ttv_5=test_out[:,1:2][25250*4:25250*5][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_5=test_out[:,1:2][25250*4:25250*5][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_5=test_out[:,1:2][25250*4:25250*5][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    Results_hh_6=test_out[:,1:2][25250*5:25250*6][:5000]
    Results_tt_6=test_out[:,1:2][25250*5:25250*6][5000:5000+19000]
    Results_tw_6=test_out[:,1:2][25250*5:25250*6][5000+19000:5000+19000+800]
    Results_tth_6=test_out[:,1:2][25250*5:25250*6][5000+19000+800:5000+19000+800+100]
    Results_ttv_6=test_out[:,1:2][25250*5:25250*6][5000+19000+800+100:5000+19000+800+100+100]
    Results_llbj_6=test_out[:,1:2][25250*5:25250*6][5000+19000+800+100+100:5000+19000+800+100+100+240]
    Results_tatabb_6=test_out[:,1:2][25250*5:25250*6][5000+19000+800+100+100+240:5000+19000+800+100+100+240+10]

    plt.rc('text', usetex=True)
    plt.rc('font', family='Time New Roman')

    logs = False

    axislabels = [ r'$DNN $']

    Yaxislabels = [ r'$DNN $']

    Bmax = 1
    Bmin = 0
    plt.xlim(0, 1)
    bins = np.linspace(Bmin, Bmax,  25)
    plt.hist(Results_hh_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_1, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    plt.hist(Results_hh_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_2, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    plt.hist(Results_hh_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_3, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    plt.hist(Results_hh_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_4, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    plt.hist(Results_hh_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_5, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()

    plt.hist(Results_hh_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='black', label= r'$h \; h$')
    plt.hist(Results_tt_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='blue', label= r'$t \; \overline{t}$')
    plt.hist(Results_tw_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, color='red', label= r'$t \; w$')
    plt.hist(Results_tth_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; h$')
    plt.hist(Results_ttv_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$t \; \overline{t} \; v$')
    plt.hist(Results_llbj_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$l \; l \; b \; j$')
    plt.hist(Results_tatabb_6, bins = bins, alpha=1, density=True, histtype='step', align = 'mid', linewidth = 1.5, log=logs, label= r'$\tau \; \tau \; b \; b$')
    plt.legend(loc=9,fontsize = 10)
    plt.xlabel(axislabels[0], fontsize = 20)
    plt.ylabel(r'$\rm{(1/\sigma) \; d \sigma / d }$' + Yaxislabels[0]    , fontsize = 20)
    plt.show()
    
    ROC_Results = open('/data/github/result/MPNN_ROC'+str(epoch), 'w')

    test_sig = 0.0
    test_Nsig, test_Nbkg = 0, 0
    for temp_cut in np.linspace(0.9, 1.0, 10000):
        pre = (test_out[:, 1]-temp_cut>=0).reshape(-1, 6).astype(int)
        tru = test_y[:]
        dev = pre-tru
        dev = abs(np.sum(dev, 1))<5
        
        hh = sum(dev[:5000])
        tt = sum(1 - dev[5000: 5000 + 19000])
        tw = sum(1 - dev[5000 + 19000: 5000 + 19000 + 800])
        tth = sum(1 - dev[5000 + 19000 + 800: 5000 + 19000 + 800 + 100])
        ttv = sum(1 - dev[5000 + 19000 + 800 + 100: 5000 + 19000 + 800 + 100 + 100])
        llbj = sum(1 - dev[5000 + 19000 + 800 + 100 + 100: 5000 + 19000 + 800 + 100 + 100 + 240])
        tatabb = sum(1 - dev[5000 + 19000 + 800 + 100 + 100 + 240: 5000 + 19000 + 800 + 100 + 100 + 240 + 10])

        Nsig = round( float( 3000*0.0214964*hh/5000*(0.8**2)/(0.7**2) ), 3)
        Nbkg = round( float( 3000*(120.907*1.596*tt/19000 + 4.38354*tw/800 + 0.15258*1.27*tth/100 + 0.157968*1.54*ttv/100 + 1.22936*llbj/240 + 0.011392*tatabb/10)*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tt = round( float( 3000*120.907*1.596*tt/19000*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tw = round( float( 3000*4.38354*tw/800*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tth = round( float( 3000*0.15258*1.27*tth/100*(0.8**2)/(0.7**2) ), 3)
        Nbkg_ttv = round( float( 3000*0.157968*1.54*ttv/100*(0.8**2)/(0.7**2) ), 3)
        Nbkg_llbj = round( float( 3000*1.22936*llbj/240*(0.8**2)/(0.7**2) ), 3)
        Nbkg_tatabb = round( float( 3000*0.011392*tatabb/10*(0.8**2)/(0.7**2) ), 3)
        SobSqrtB = round( float( IndiLikeRatioDis(float( Nsig ),float( Nbkg ) )  ) ,   3 )
        ROC_Results.write(str(Nsig) + ' ' + str(Nbkg) + ' ' + str(SobSqrtB) + ' ' + str(Nbkg_tt) + ' ' + str(Nbkg_tw) + ' ' + str(Nbkg_tth) + ' ' + str(Nbkg_ttv) + ' ' + str(Nbkg_llbj) + ' ' + str(Nbkg_tatabb)   ) 
        ROC_Results.write('\n')

    ROC_Results.close()    

    ROC_Results= np.loadtxt('/data/github/result/MPNN_ROC'+str(epoch))

    SB=[]
    hh=[]
    tt=[]
    tw=[]
    tth=[]
    ttv=[]
    llbj=[]
    tatabb=[]

    for n in range(len(ROC_Results)):  
        SB.append(ROC_Results[n][2])
        hh.append(ROC_Results[n][0])
        tt.append(ROC_Results[n][3])
        tw.append(ROC_Results[n][4])
        tth.append(ROC_Results[n][5])
        ttv.append(ROC_Results[n][6])
        llbj.append(ROC_Results[n][7])
        tatabb.append(ROC_Results[n][8])

    plt.plot(hh,SB, color='r', label='Significance')
    plt.xlabel(r'$ N_s $', fontsize=20)
    plt.ylabel(r'Significance', fontsize=20)
    plt.legend(loc='best', fontsize=15)
    plt.show()

    j=SB.index(max(SB))

    print('\nsignificance: {:.3f} hh: {:.3f} tt: {:.3f} tw: {:.3f} tth: {:.3f} ttv: {:.3f} llbj: {:.3f} tatabb: {:.3f} \n'.format(SB[j], hh[j], tt[j], tw[j], tth[j], ttv[j], llbj[j], tatabb[j]))
    
    if epoch % 1 == 0:
        sig = max(SB)
        best_sig = max(best_sig,sig)           

    end=time.time()

    print('* Best Significance : {:.3f} *'.format(best_sig))

    print('Epoch time: {:.2f} mins'.format((end-start)/60))
    print('='*69)      